In [2]:
from flask import Flask, request, render_template_string
import threading
import ollama

In [3]:
app = Flask(__name__)

In [4]:
# HTML : zone de texte + menu déroulant pour la langue du résumé

In [5]:
TEMPLATE = """
<!doctype html>
<html lang="fr">
  <head>
    <meta charset="utf-8">
    <title>Analyse de Texte</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600&display=swap" rel="stylesheet">
    <style>
      body {
        font-family: 'Inter', sans-serif;
        background: linear-gradient(to right, #eef2f3, #8e9eab);
        min-height: 100vh;
        display: flex;
        align-items: center;
        justify-content: center;
      }
      .card {
        border-radius: 1rem;
        box-shadow: 0 8px 20px rgba(0, 0, 0, 0.1);
      }
      h1 {
        font-weight: 600;
      }
    </style>
  </head>
  <body>
    <div class="container">
      <div class="card p-4 p-md-5 my-5">
        <h1 class="text-center mb-4">📝 Analyse de Texte Intelligent</h1>
        <form method="post">
          <div class="mb-3">
            <label for="text" class="form-label">Texte à analyser :</label>
            <textarea class="form-control" id="text" name="text" rows="8"
                      placeholder="Entrez votre texte ici...">{{ request.form.text or '' }}</textarea>
          </div>

          <div class="mb-3">
            <label for="lang" class="form-label">Langue de sortie :</label>
            <select name="lang" id="lang" class="form-select">
              <option value="fr" {% if request.form.get('lang') == 'fr' %}selected{% endif %}>Français</option>
              <option value="en" {% if request.form.get('lang') == 'en' %}selected{% endif %}>Anglais</option>
              <option value="ar" {% if request.form.get('lang') == 'ar' %}selected{% endif %}>Arabe</option>
            </select>
          </div>

          <div class="mb-3">
            <label for="task" class="form-label">Action à effectuer :</label>
            <select name="task" id="task" class="form-select">
              <option value="summary" {% if request.form.get('task') == 'summary' %}selected{% endif %}>Résumé</option>
              <option value="keywords" {% if request.form.get('task') == 'keywords' %}selected{% endif %}>Mots-clés</option>
              <option value="category" {% if request.form.get('task') == 'category' %}selected{% endif %}>Catégorie</option>
              <option value="simplify" {% if request.form.get('task') == 'simplify' %}selected{% endif %}>Simplification</option>
              <option value="facts" {% if request.form.get('task') == 'facts' %}selected{% endif %}>Faits clés</option>
              <option value="title" {% if request.form.get('task') == 'title' %}selected{% endif %}>Titre</option>
              <option value="rewrite" {% if request.form.get('task') == 'rewrite' %}selected{% endif %}>Reformulation</option>
              <option value="tone" {% if request.form.get('task') == 'tone' %}selected{% endif %}>Analyse du ton</option>
              <option value="sentiment" {% if request.form.get('task') == 'sentiment' %}selected{% endif %}>Analyse du sentiment</option>
              <option value="question_generation" {% if request.form.get('task') == 'question_generation' %}selected{% endif %}>Génération de questions</option>
              <option value="text_to_speech" {% if request.form.get('task') == 'text_to_speech' %}selected{% endif %}>Texte vers parole</option>
                <option value="grammar_correction" {% if request.form.get('task') == 'grammar_correction' %}selected{% endif %}>Correction grammaticale</option>
                <option value="text_completion" {% if request.form.get('task') == 'text_completion' %}selected{% endif %}>Complétion de texte</option>
                <option value="code_generation" {% if request.form.get('task') == 'code_generation' %}selected{% endif %}>Génération de code</option>

            </select>
          </div>

          <div class="text-center">
            <button type="submit" class="btn btn-primary px-5">🚀 Lancer</button>
          </div>
        </form>

        {% if result %}
          <div class="mt-4">
            <h4>Résultat :</h4>
            <div class="alert alert-success p-3 shadow-sm">
              {{ result }}
            </div>
          </div>
        {% endif %}
        {% if response_time %}
  <div class="mt-2 text-muted">
    <small>⏱ Temps de réponse : {{ '%.2f'|format(response_time) }} secondes</small>
  </div>
{% endif %}

        
      </div>
    </div>

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/js/bootstrap.bundle.min.js"></script>
  </body>
</html>
"""


In [6]:
# Fonction de résumé : résume dans la langue choisie

In [7]:
def process_text(text: str, lang_choice: str, task: str) -> str:
    """
    Traite le texte selon la tâche choisie ('summary', 'keywords', etc.)
    et la langue ('en', 'fr', 'ar'), en utilisant le modèle Ollama.
    Retourne le contenu généré.
    """
    # Mapping du code de langue vers le libellé
    lang_map = {"en": "English", "fr": "French", "ar": "Arabic"}
    target_lang = lang_map.get(lang_choice, "French")  # Par défaut : French

    # Prompt système général
    system_prompt = (
        f"You are a helpful assistant. Always reply in {target_lang}. "
        "Translate if necessary so the response is in that language."
    )

    # Dictionnaire des prompts par tâche
    prompts = {
        'summary': (
            f"You are an expert summarization assistant. Your goal is to create clear, concise, and faithful summaries "
            f"of user-provided texts. Always write the summary in {target_lang}. If the original text is in another language, "
            f"translate it before summarizing. Make sure the summary captures the essential information without unnecessary details. "
            f"Use simple and professional language. Avoid repeating the original text. Keep the summary coherent and easy to understand.\n\n"
            f"Text:\n{text}"
        ),
        'keywords': f"Extract 3 to 5 keywords or key themes from this text in {target_lang}:\n{text}",
        'category': f"Classify this text into a topic category (e.g., health, politics, education) in {target_lang}:\n{text}",
        'simplify': f"Rewrite this text in a simple and accessible language in {target_lang}:\n{text}",
        'facts':    f"Extract factual information (dates, numbers, names) from this text in {target_lang}:\n{text}",
        'title':    f"Generate a short, informative title for this text in {target_lang}:\n{text}",
        'rewrite':  f"Rewrite this text in a journalistic tone in {target_lang}:\n{text}",
        'tone':     f"Analyze the tone and bias of this text in {target_lang}:\n{text}",
        'sentiment': f"Analyze the sentiment of this text in {target_lang} (positive, negative, neutral):\n{text}",
        'question_generation': f"Generate 3 relevant quiz questions and answers from this text in {target_lang}:\n{text}",
        'text_to_speech': f"Convert this text to speech in {target_lang}. Provide a phonetic transcription or description:\n{text}",
        'grammar_correction': f"Correct grammar and spelling errors in this text in {target_lang}:\n{text}",
        'text_completion': f"Complete the following text naturally in {target_lang}:\n{text}",
        'code_generation': f"Generate a code snippet based on this description in {target_lang}:\n{text}"
    }

    # Récupérer le prompt correspondant à la tâche
    user_prompt = prompts.get(task, prompts['summary'])  # fallback sur summary

    # Messages pour Ollama
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # Appel au modèle
    response = ollama.chat(model="llama3.2", messages=messages)

    return response["message"]["content"]


In [8]:
# Route Flask

In [9]:
import time

@app.route("/", methods=["GET", "POST"])
def index():
    result = None
    response_time = None
    if request.method == "POST":
        text = request.form.get("text", "")
        lang_choice = request.form.get("lang", "fr")
        task = request.form.get("task", "summary")
        if text.strip():
            try:
                start_time = time.time()  # début du chrono
                result = process_text(text, lang_choice, task)
                end_time = time.time()    # fin du chrono
                response_time = end_time - start_time
            except Exception as e:
                result = f"❌ Erreur lors du traitement : {e}"
        else:
            result = "❗ Veuillez entrer un texte à analyser."

    return render_template_string(TEMPLATE, result=result, response_time=response_time)


In [ ]:
# Lancement (thread pour ne pas bloquer quand on exécute le script)
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Jun/2025 19:15:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2025 19:15:37] "GET /favicon.ico HTTP/1.1" 404 -


🔍 1. Analyse thématique
Identifier les sujets principaux ou mots-clés du texte.

Exemple : ["économie", "environnement", "transition énergétique"]

🧠 Prompt additionnel :

"Also identify 3 to 5 key themes or keywords that represent the core of the text."

🗂 2. Catégorisation automatique
Classer le texte dans une catégorie : politique, santé, éducation, technologie, etc.

Utile pour des systèmes de tri ou filtres automatiques.

🧠 Prompt additionnel :

"Classify this text into a topic category (e.g., health, education, politics, etc.)."

✍️ 3. Réécriture simplifiée
Réécriture du texte en langage plus simple, accessible ou pédagogique.

Parfait pour un public non spécialiste (ex : enfants, non-experts).

🧠 Prompt :

"Rewrite this text in a simplified, accessible way for a general audience, in {target_lang}."

🧾 4. Extraction de faits ou données clés
Dégager les chiffres importants, noms, dates, faits.

Très utile pour des articles scientifiques, juridiques ou techniques.

🧠 Prompt :

"Extract any important facts, figures, or dates mentioned in the text."

💡 5. Génération d’un titre ou d’un slogan
Créer un titre accrocheur ou une phrase résumée.

Parfait pour des résumés destinés à être affichés dans une interface.

🧠 Prompt :

"Suggest a short, informative title for this text in {target_lang}."

📚 6. Reformulation personnalisée
Demander un résumé :

En style académique

En style journalistique

En style convivial / conversationnel

🧠 Prompt :

"Summarize this text in a journalistic style in {target_lang}."

🔄 7. Détection de biais ou ton
Analyse si le texte est objectif, persuasif, ironique, etc.

Cela peut être utile pour la modération ou la recommandation.

🧠 Prompt :

"Analyze the tone and possible bias of the text."

